This is a general purpose workbook for training models.


In [1]:
#@title Package imports etc
#https://docs.databricks.com/en/_extras/notebooks/source/deep-learning/tune-classification-model-hugging-face-transformers.html
#https://github.com/zo-fi/slo_wsd_ZFMA
#https://stackoverflow.com/questions/76448287/how-can-i-solve-importerror-using-the-trainer-with-pytorch-requires-accele
! pip install -U accelerate
! pip install -U transformers

!pip install evaluate
!pip install datasets

import pandas as pd
import xml.etree.ElementTree as ET
import re
import os

from google.colab import drive
drive.mount('/content/drive/')
import numpy as np
from tqdm import tqdm
import seaborn as sns
import nltk
import json
#import classla
from tqdm import tqdm
from tqdm.gui import tqdm as tqdm_gui
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy
import evaluate
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

import torch
cuda_test = torch.cuda.is_available()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
import random
random.seed(101)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
#loading the data properly

rwg = pd.read_csv("drive/MyDrive/projs/rep_sp/data/rwg_processed.csv")
riq = pd.read_csv("drive/MyDrive/projs/rep_sp/data/riqua_processed.csv")

rwg_train = rwg[rwg["split"] == "train"]
rwg_dev = rwg[rwg["split"] == "dev"]
riq_train = riq[riq["split"] == "train"]
riq_dev = riq[riq["split"] == "dev"]

train_df = pd.concat([rwg_train, riq_train])
dev_df = pd.concat([riq_train, riq_dev])

"sents", "quote_tag"
train_df.rename(columns = {"sents":"example", "quote_tag":"label"}, inplace = True)
train_df["label"] = train_df.label.apply(int)

dev_df.rename(columns = {"sents":"example", "quote_tag":"label"}, inplace = True)
dev_df["label"] = dev_df.label.apply(int)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(dev_df)
#test_dataset = Dataset.from_pandas(test_df)

datasets = DatasetDict({"train" : train_dataset, "validation" : val_dataset})#, "test": test_dataset

#train_df = pd.concat([, )
#val_df = pd.concat(rwg[rwg["split"] == "val"], riq[riq["split"] == "val"])

#data = "drive/MyDrive/projs/rep_sp/rwg_processed.csv"
#df = pd.read_csv(data)
#del df["index"]

#train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify = df[["fictional", "label"]])
#train_df, val_df = train_test_split(df, test_size=0.125, random_state=101, stratify = df[["fictional", "label"]])


In [ ]:
train_df.label.apply(type)

0        <class 'float'>
1        <class 'float'>
2        <class 'float'>
3        <class 'float'>
4        <class 'float'>
              ...       
38605    <class 'float'>
38606    <class 'float'>
38607    <class 'float'>
38608    <class 'float'>
38609    <class 'float'>
Name: label, Length: 46064, dtype: object

In [3]:
#@title Load model, tokenizer
tgt_model = "google-bert/bert-base-multilingual-cased" #"FacebookAI/xlm-roberta-base" #

model = AutoModelForSequenceClassification.from_pretrained("zo-fi/rep_sp_untrained_polnear_bert", return_dict=True, num_labels=2)
freeze_list = ["bert.encoder.layer.{}".format(str(i)) for i in range(0, 8)]
for name, param in model.named_parameters():
    if any([name.startswith(ban_param) for ban_param in freeze_list]):
        param.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained(tgt_model)
def encode(examples):
  out = tokenizer(examples["example"], truncation = True, padding = "max_length") #, max_length = MAX_LEN
  return out

encoded_dataset = datasets.map(encode, batched=True)
encoded_dataset.set_format(columns=['attention_mask', 'input_ids', 'token_type_ids', 'label'])
#label2id=label2id, id2label=id2label?


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/46064 [00:00<?, ? examples/s]

Map:   0%|          | 0/31586 [00:00<?, ? examples/s]

In [ ]:
#!pip install accelerate -U

metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
def compute_metrics(predic_obj):
    logits = predic_obj.predictions
    labels = predic_obj.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=predictions, references=labels)

training_output_dir = "drive/MyDrive/projs/rep_sp/train_dump/"
training_args = TrainingArguments(
    output_dir = training_output_dir,
    evaluation_strategy="epoch",
    num_train_epochs = 2,#
    logging_dir='./logs',      # directory for storing logs
    gradient_accumulation_steps = 16)

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer),
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model=model#, callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,0.250671


In [ ]:
model.eval()
preds = trainer.predict(encoded_dataset["test"])
#compute_metrics

#metrics={'test_loss': 0.3038247525691986,
# 	Training Loss 	Validation Loss
#0 	No log 	0.320669
#1 	No log 	0.317925
#TrainOutput(global_step=328, training_loss=0.35928954147711034, metrics={'train_runtime': 3263.6252, 'train_samples_per_second': 12.886, 'train_steps_per_second': 0.101, 'total_flos': 1.10454021040128e+16, 'train_loss': 0.35928954147711034, 'epoch': 1.9961962723469})

In [ ]:
#os.mkdir("./drive/MyDrive/projs/rep_sp/models")
print(tgt_model)

In [ ]:
model.save_pretrained("./drive/MyDrive/projs/rep_sp/models/trained_polnear_mbert")
#os.listdir("./drive/MyDrive/projs/rep_sp/models")

In [ ]:
#Data exploration
#a lot of fictional data actually, maybe separate testig would be a good idea
#I don't really wanna throw away the data
df.columns
df.fictional.value_counts()
df[df.fictional == "no"].quote_tag.value_counts()
df[df.fictional == "yes"].quote_tag.value_counts()
df_train.quote_tag.value_counts()
df_test.quote_tag.value_counts()